In [80]:
import numpy as np
import pandas as pd
import time
from datetime import date, timedelta

from nba_api.stats.library.parameters import PerModeDetailed
from requests.exceptions import ReadTimeout

In [61]:
#building the pregame table

df= pd.read_csv(r"C:\Users\ttjrb\OneDrive\Desktop\University Important\PROJECTS\NBA data analyzed\database\schedule_info.csv", dtype={"GAME_ID": str})
df["GAME_ID"]= "00"+df["GAME_ID"]
df["TIME"]= df["TIME"].str[:2]
df= pd.get_dummies(df, columns=['DAY'], dtype=int)
df

,GAME_ID,TEAM1,TEAM2,DATE,TIME,NATIONAL_TV,DAY_Friday,DAY_Monday,DAY_Saturday,DAY_Sunday,DAY_Thursday,DAY_Tuesday,DAY_Wednesday
0,0022401186,ATL,ORL,2025-04-13,13,0,0,0,0,1,0,0,0
1,0022401188,BKN,NYK,2025-04-13,13,0,0,0,0,1,0,0,0
2,0022401195,UTA,MIN,2025-04-13,15,0,0,0,0,1,0,0,0
3,0022401194,MEM,DAL,2025-04-13,15,0,0,0,0,1,0,0,0
4,0022401196,OKC,NOP,2025-04-13,15,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,0022400070,MEM,UTA,2024-10-23,21,0,0,0,0,0,0,0,1
1225,0022400067,CLE,TOR,2024-10-23,19,0,0,0,0,0,0,0,1
1226,0022400069,CHI,NOP,2024-10-23,20,0,0,0,0,0,0,0,1
1227,0022400061,BOS,NYK,2024-10-22,19,1,0,0,0,0,0,1,0


In [82]:

#building the pregame table helper, season stats of each team

players= pd.read_csv(r"C:\Users\ttjrb\OneDrive\Desktop\University Important\PROJECTS\NBA data analyzed\database\playerstats24.csv")
players["Team"]=players["Team"].astype(str)
players.loc[players["Team"] == "PHO", "Team"] = "PHX"
players.loc[players["Team"] == "CHO", "Team"] = "CHA"
players.loc[players["Team"] == "BRK", "Team"] = "BKN"

#RIVALRIES
teams= df["TEAM1"].unique()
teams_id=["1610612737","1610612751","1610612762","1610612763","1610612760","1610612758","1610612749","1610612738","1610612757",
 "1610612743","1610612764","1610612744","1610612755","1610612739","1610612761","1610612746","1610612754","1610612747",
 "1610612765","1610612741","1610612756","1610612752","1610612740","1610612750","1610612766","1610612748","1610612742",
 "1610612753","1610612759","1610612745"]

rivals=[['BOS','MIA','NYK'],['NYK'],['HOU','DEN'],['GSW','OKC'],['DAL','SAS'],['LAL', 'GSW'],
        ['BOS','CHI'],['LAL', 'PHI', 'NYK', 'MIA'],['OKC', 'LAL'],['UTA', 'LAL'],['CLE'],['CLE', 'SAC', 'LAL'],
        ['BOS', 'TOR'],['GSW', 'WAS'],['BOS', 'PHI'],['LAL', 'GSW'],['NYK', 'MIA'], ['BOS', 'LAC', 'SAC', 'GSW'],
        ['CHI', 'LAL'], ['DET', 'NYK', 'MIL'],['SAS', 'LAL'],['BOS', 'CHI', 'MIA', 'IND', 'BKN'],['UTA', 'MEM'],
        ['DEN', 'DAL'], ['MIA'], ['NYK', 'ORL', 'BOS'], ['SAS', 'HOU', 'OKC'], ['MIA'], ['DAL', 'LAL', 'HOU'], ['DAL', 'SAS']]
teams_data= pd.DataFrame({"TEAM": teams, "TEAM_ID":teams_id, "RIVALS": rivals})

#LEADERS
perf_dict= {team:0 for team in teams}
    #pts
pts=(players.sort_values(by="PTS",ascending=False)).head(30)
for team in pts["Team"]:
    if team=="2TM":
        continue
    perf_dict[team]+=1
    #ast
pts=(players.sort_values(by="AST",ascending=False)).head(30)
for team in pts["Team"]:
    if team=="2TM":
        continue
    perf_dict[team]+=1
    #reb
pts=(players.sort_values(by="TRB",ascending=False)).head(30)
for team in pts["Team"]:
    if team=="2TM":
        continue
    perf_dict[team]+=1
    #blk
pts=(players.sort_values(by="BLK",ascending=False)).head(30)
for team in pts["Team"]:
    if team=="2TM":
        continue
    perf_dict[team]+=1
    #stl
pts=(players.sort_values(by="STL",ascending=False)).head(30)
for team in pts["Team"]:
    if team=="2TM":
        continue
    perf_dict[team]+=1
    #3pm
pts=(players.sort_values(by="3P",ascending=False)).head(30)
for team in pts["Team"]:
    if team=="2TM":
        continue
    perf_dict[team]+=1

teams_data["LEADERS"]=perf_dict.values()

#STATS
from nba_api.stats.endpoints import leaguedashteamstats
    # Get team stats for the 2024-25 regular season
team_stats = leaguedashteamstats.LeagueDashTeamStats(
    season='2024-25',
    season_type_all_star='Regular Season',
    per_mode_detailed='PerGame'
)

    # Convert to DataFrame
stats = team_stats.get_data_frames()[0]

    #get rows I want
stats= stats.drop(["TEAM_NAME","GP","W","L","MIN"], axis=1)
stats= stats.drop(columns=stats.columns[23:])

    #append to teams table
teams_data["TEAM_ID"]= teams_data["TEAM_ID"].astype(int)
teams_data= teams_data.merge(stats,on=["TEAM_ID"])


teams_data


,TEAM,TEAM_ID,RIVALS,LEADERS,W_PCT,FGM,FGA,FG_PCT,FG3M,FG3A,...,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS
0,ATL,1610612737,"[BOS, MIA, NYK]",6,0.460,42.5,90.6,0.469,13.5,37.8,...,44.2,29.0,15.7,9.6,5.1,4.9,19.3,19.2,116.3,-1.3
1,BKN,1610612751,[NYK],2,0.333,37.3,85.4,0.436,13.6,39.4,...,40.8,24.8,15.2,8.0,4.3,5.4,20.8,19.1,104.3,-6.8
2,UTA,1610612762,"[HOU, DEN]",5,0.244,39.4,87.2,0.452,13.5,38.8,...,44.9,24.9,17.4,6.9,4.6,6.3,19.1,19.6,109.7,-8.7
3,MEM,1610612763,"[GSW, OKC]",5,0.593,43.8,92.0,0.477,13.4,36.9,...,46.3,27.6,15.4,9.1,5.5,5.6,21.2,20.5,119.3,5.0
4,OKC,1610612760,"[DAL, SAS]",8,0.789,43.3,91.1,0.476,14.3,38.5,...,44.2,26.2,12.1,10.1,5.5,4.8,19.6,18.2,117.4,11.3
5,SAC,1610612758,"[LAL, GSW]",5,0.483,42.4,88.8,0.477,12.4,34.7,...,43.6,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4
6,MIL,1610612749,"[BOS, CHI]",8,0.552,41.4,85.8,0.483,13.9,36.4,...,43.1,25.1,13.6,7.3,4.7,3.9,18.0,18.7,113.9,1.3
7,BOS,1610612738,"[LAL, PHI, NYK, MIA]",9,0.724,41.2,89.2,0.462,17.4,47.5,...,44.8,25.9,12.0,7.1,5.5,3.7,16.2,17.6,115.1,8.6
8,POR,1610612757,"[OKC, LAL]",5,0.448,40.0,89.6,0.446,12.7,37.8,...,44.7,23.5,16.0,8.3,5.4,5.6,19.0,19.4,109.4,-2.9
9,DEN,1610612743,"[UTA, LAL]",11,0.598,44.3,88.6,0.500,11.8,31.8,...,45.3,30.1,14.2,7.9,4.8,5.3,17.7,19.1,118.4,3.6


In [35]:
#teams_data.to_csv("2024_leaguestats.csv", index=False)

In [83]:
finaldf=pd.merge(df,teams_data, left_on='TEAM1', right_on='TEAM')
finaldf=pd.merge(finaldf,teams_data, left_on='TEAM2', right_on='TEAM')
finaldf["rivalry"] = finaldf.apply(lambda row: 1 if row['TEAM2'] in row['RIVALS_x'] or row['TEAM1'] in row['RIVALS_y'] else 0,axis=1)

In [85]:
finaldf= finaldf.drop(['TEAM_x','TEAM_y','RIVALS_x','RIVALS_y'], axis=1)
finaldf.rename(columns={'TEAM1':'TEAM_x', 'TEAM2':'TEAM_y'}, inplace=True)
finaldf

,GAME_ID,TEAM_x,TEAM_y,DATE,TIME,NATIONAL_TV,DAY_Friday,DAY_Monday,DAY_Saturday,DAY_Sunday,...,AST_y,TOV_y,STL_y,BLK_y,BLKA_y,PF_y,PFD_y,PTS_y,PLUS_MINUS_y,rivalry
0,0022401186,ATL,ORL,2025-04-13,13,0,0,0,0,1,...,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0,0
1,0022401149,ATL,ORL,2025-04-08,19,0,0,0,0,0,...,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0,0
2,0022400756,ATL,ORL,2025-02-10,19,0,0,1,0,0,...,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0,0
3,0022400305,BKN,ORL,2024-12-01,15,0,0,0,0,1,...,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0,0
4,0022400798,MEM,ORL,2025-02-21,19,0,1,0,0,0,...,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,0022400448,DAL,SAC,2024-12-30,22,1,0,1,0,0,...,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4,0
1225,0022401074,ORL,SAC,2025-03-29,17,0,0,0,1,0,...,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4,0
1226,0022400337,SAS,SAC,2024-12-06,20,0,1,0,0,0,...,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4,0
1227,0022400313,SAS,SAC,2024-12-01,21,0,0,0,0,1,...,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4,0


In [86]:
finaldf.rename(columns={'rivalry':'RIVALRY'}, inplace=True)

col = finaldf.columns.tolist() # Get list of columns
col.insert(2, col.pop(col.index("TEAM_ID_x"))) # Move "MyColumn" to position 2
col.insert(4, col.pop(col.index("TEAM_ID_y")))
col.insert(7, col.pop(col.index("RIVALRY")))
finaldf = finaldf[col]

finaldf

,GAME_ID,TEAM_x,TEAM_ID_x,TEAM_y,TEAM_ID_y,DATE,TIME,RIVALRY,NATIONAL_TV,DAY_Friday,...,REB_y,AST_y,TOV_y,STL_y,BLK_y,BLKA_y,PF_y,PFD_y,PTS_y,PLUS_MINUS_y
0,0022401186,ATL,1610612737,ORL,1610612753,2025-04-13,13,0,0,0,...,41.4,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0
1,0022401149,ATL,1610612737,ORL,1610612753,2025-04-08,19,0,0,0,...,41.4,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0
2,0022400756,ATL,1610612737,ORL,1610612753,2025-02-10,19,0,0,0,...,41.4,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0
3,0022400305,BKN,1610612751,ORL,1610612753,2024-12-01,15,0,0,0,...,41.4,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0
4,0022400798,MEM,1610612763,ORL,1610612753,2025-02-21,19,0,0,1,...,41.4,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,0022400448,DAL,1610612742,SAC,1610612758,2024-12-30,22,0,1,0,...,43.6,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4
1225,0022401074,ORL,1610612753,SAC,1610612758,2025-03-29,17,0,0,0,...,43.6,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4
1226,0022400337,SAS,1610612759,SAC,1610612758,2024-12-06,20,0,0,1,...,43.6,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4
1227,0022400313,SAS,1610612759,SAC,1610612758,2024-12-01,21,0,0,0,...,43.6,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4


In [105]:
postgame= pd.read_csv(r"C:\Users\ttjrb\OneDrive\Desktop\University Important\PROJECTS\NBA data analyzed\database\2024_postgame.csv", dtype={"GAME_ID": str})
postgame= postgame[["GAME_ID","EI"]]
pregame= finaldf.merge(postgame, on="GAME_ID")

In [106]:
pregame

,GAME_ID,TEAM_x,TEAM_ID_x,TEAM_y,TEAM_ID_y,DATE,TIME,RIVALRY,NATIONAL_TV,DAY_Friday,...,AST_y,TOV_y,STL_y,BLK_y,BLKA_y,PF_y,PFD_y,PTS_y,PLUS_MINUS_y,EI
0,0022401186,ATL,1610612737,ORL,1610612753,2025-04-13,13,0,0,0,...,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0,0.189419
1,0022401149,ATL,1610612737,ORL,1610612753,2025-04-08,19,0,0,0,...,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0,0.219147
2,0022400756,ATL,1610612737,ORL,1610612753,2025-02-10,19,0,0,0,...,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0,0.335673
3,0022400305,BKN,1610612751,ORL,1610612753,2024-12-01,15,0,0,0,...,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0,0.300246
4,0022400798,MEM,1610612763,ORL,1610612753,2025-02-21,19,0,0,1,...,23.0,14.3,8.9,5.9,4.3,20.0,20.2,105.1,0.0,0.333578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,0022400448,DAL,1610612742,SAC,1610612758,2024-12-30,22,0,1,0,...,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4,0.105225
1225,0022401074,ORL,1610612753,SAC,1610612758,2025-03-29,17,0,0,0,...,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4,0.061527
1226,0022400337,SAS,1610612759,SAC,1610612758,2024-12-06,20,0,0,1,...,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4,0.118711
1227,0022400313,SAS,1610612759,SAC,1610612758,2024-12-01,21,0,0,0,...,26.1,13.7,7.6,4.4,4.2,19.1,18.6,114.0,0.4,0.441388


In [108]:
pregame.sort_values(by=['DATE'],ascending=False).to_csv("2024_pregame.csv", index=False)